In [ ]:
import os
import nibabel as nib
import glob
from PIL import Image
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt 
import cv2 as cv
from IPython.display import clear_output
from random import seed
import random
import shutil

In [ ]:
def get_images(row, dataset_type ,side):
    randid = row["rand_ID"]
    p_id = row["subjectID_wo_17"]
    pid = str(int(p_id)).zfill(2)
    rand_id = str(int(randid)).zfill(2)
    
    if(side=="right"):
        end = row["head1stR"]
        start = row["lastR"]
        slide_id = rand_id+"cr"
    else:
        start = row["lastL"]
        end = row["head1stL"]
        slide_id = rand_id+"cl"
    path = "/data/scratch/chirag24/HIA/data/"+pid+"/"+slide_id+"/"
    list_dir = os.listdir(path) # dir is your directory path
    slide_id_name = slide_id +"-s"
    #path = path+ slide_id_name
    des_path = "/data/scratch/chirag24/HIA/dataset/"+dataset_type

    no_files = len(list_dir)
    for x in range(no_files):
        img_name = slide_id_name + str(x)+".png"
        source = path + img_name
        if(x >= start and x<= end):
            #move to hippocampus
            destination = des_path + "/hippocampus/"
        else:
            #move to non-hippocampus
            destination = des_path + "/non_hippocampus/"
        os.system("mkdir -p "+ destination)
        shutil.copyfile(source, destination+img_name)

In [3]:
def get_csv_rows(sub_id):
    path = "/data/scratch/chirag24/HIA/"
    df = pd.read_csv(path+"slide_data.csv", skiprows=4)
    var = float('nan')
    df.dropna(how='all')
    df = df[df['subjectID_wo_17'] == sub_id]
    return df
    

In [4]:
train_count = 0
testCount = 0
Tcount = 21
train_percent = 70
percent = (Tcount-1) * 70 /100
for x in range(Tcount):
    if(x==17):
        continue
    value = random.randint(0, Tcount)
     
    if value%2 == 0 and train_count < percent:  
        train_count += 1
        dataset_type = "train"
    elif testCount < (Tcount -1 - percent):
        testCount += 1
        dataset_type = "test"
    else:
        train_count += 1
        dataset_type = "train"
    df = get_csv_rows(x)
    for index, row in df.iterrows():
        get_images(row, dataset_type,"left")
        get_images(row, dataset_type,"right")
    